# Figure 7

The following script was used to generate the graphs of Figure 7 of the article "Behavioral Diversity Generation in Autonomous Exploration Through Reuse of Past Experience" by Fabien C. Y. Benureau and Pierre-Yves Oudeyer.

The [full code is available](http://fabien.benureau.com/code/frontiers2016.html) and is distributed under the [Open Science License](http://fabien.benureau.com/openscience.html). For any questions, remarks or difficulties running this code, contact [fabien.benureau@gmail.com](mailto:fabien.benureau@gmail.com).

In [ ]:
import random

import scipy.stats
from bokeh import plotting
import pyprind # progress bars

import explorers
import environments

import dotdot
import exs
import envs
import factored
import graphs


random.seed(0)
graphs.output_notebook()

In [ ]:
DIM = 20    # number of joints
N   = 20000 # size of the total population
M   = 1000  # maximum size of subsets whose diversity is compared

env1 = environments.Environment.create(envs.catalog['kin{}_150'.format(DIM)])
env2 = environments.Environment.create(envs.catalog['kin{}_150_0.9'.format(DIM)])

m_signals = [environments.tools.random_signal(env1.m_channels) for _ in range(N)]

# creating comparative s_vectors
s_vectors1, s_vectors2 = [], []
for m_signal in m_signals:
    s_vectors1.append(environments.tools.to_vector(env1.execute(m_signal)['s_signal'], env1.s_channels))
    s_vectors2.append(environments.tools.to_vector(env2.execute(m_signal)['s_signal'], env2.s_channels))

In [ ]:
# this will take a few minutes.
def compute_diversities(s_vectors1, s_vectors2):
    """Compute the diversity of random subsets of the sensory vectors"""
    divs = []
    progress_bar = pyprind.ProgBar(M-1)

    assert len(s_vectors1) == len(s_vectors2)

    
    for k in range(1, M, 1):
        for _ in range(1):
            pop_idx = random.sample(range(len(s_vectors1)), k)
            pop1 = [s_vectors1[i] for i in pop_idx]
            pop2 = [s_vectors2[i] for i in pop_idx]

            div1 = factored.run_coverage(0.05, pop1)
            div2 = factored.run_coverage(0.05, pop2)
            divs.append((div1, div2))
        progress_bar.update()
            
    return divs

divs = compute_diversities(s_vectors1, s_vectors2)

In [ ]:
# Peason correlation
print(scipy.stats.pearsonr([d[0] for d in divs], [d[1] for d in divs]))

In [ ]:
fig = plotting.figure(title='diversity correlation_{}'.format(DIM), x_range=[0, 1.4], y_range=[0, 1.4])
fig.scatter([d[0] for d in divs], [d[1] for d in divs],
            radius=1.2, radius_units='screen', line_color=None, fill_alpha=0.5)
graphs.show(fig)

## Provenance Data

In [ ]:
import provenance
prov_data = provenance.planar_arms()
print(prov_data.message())